## 카카오페이 크롤링

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

def scroll_down(driver):
    driver.execute_script('document.getElementsByClassName("fysCi")[0].scrollTo(0, 999999999999)') # 그 모든 리뷰 나오는 작은 창에서 스크롤을 내리도록 해야되기 때문에, 그 작은 창의 HTML 요소를 알려주도록 합니다.
    time.sleep(1)

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 현재 페이지의 댓글을 크롤링
channel_url = 'https://play.google.com/store/apps/details?id=com.kakaopay.app&hl=ko&gl=US'

# implicitly_wait() 메서드로 기다리는 시간 설정

driver.implicitly_wait(5)

# 채널 페이지로 이동

driver.get(channel_url)

data = pd.DataFrame(data=[], columns=['날짜', '리뷰', '별점'])


# 리뷰 모두 보기 버튼 클릭

button = driver.find_element(By.CSS_SELECTOR, 'div.Jwxk6d span.VfPpkd-vQzf8d') 
button.click()
time.sleep(1)

# 리뷰 화면 창 나왔으면, 스크롤 다섯번 내리기
for i in range(120):
    # driver.find_element(By.CSS_SELECTOR, "div.odk6He").send_keys(Keys.END)
    scroll_down(driver)
    time.sleep(5)

# 이제 리뷰들 수집

dates = driver.find_elements(by=By.XPATH, value='//span[@class="bp9Aid"]')
reviews = driver.find_elements(by=By.XPATH, value='//div[@class="h3YV2d"]')
stargrades = driver.find_elements(by=By.XPATH, value='//div[@class="iXRFPc"]')


for i in range(len(reviews)):
    tmp = []
    tmp.append(dates[i].text)  # 날짜 저장하기
    tmp.append(reviews[i].text)  # 리뷰 저장하기
    tmp.append(stargrades[i].get_attribute('aria-label'))  # 평점 저장하기
    tmp = pd.DataFrame(data=[tmp], columns=data.columns)
    data = pd.concat([data, tmp])

print("카카오페이 앱 리뷰 수집 완료")

data.reset_index(inplace=True, drop=True)
print(data.to_string())

In [ ]:
data

In [ ]:
data.head(10)

In [ ]:
data2=data.iloc[4:,:]
data2

In [ ]:
data2['month']=data2['날짜'].str.split(' ').str[0]
data2.head(5)

In [ ]:
import re 

data2['별점'] = data2['별점'].apply(lambda x : x[5:])

m = re.compile('[0-9][\.0-9]*') #정규표현식

data2['별점'] = data2['별점'].apply(lambda x : m.findall(x)[0])
data2['별점']

In [ ]:
data2.groupby('month').count()

In [ ]:
data2.to_csv('C:/Users/junbong heo/OneDrive/바탕 화면/kakaopay_review_scroll120.csv',index=False,encoding='utf-8-sig')